In [1]:
import requests
import os
import csv
import pandas as pd
import requests
import numpy as np
from io import BytesIO
from colorthief import ColorThief
from bs4 import BeautifulSoup

In [2]:
url = 'https://mrmrsenglish.com/robots.txt'
response = requests.get(url)
print(response.text) 

# START YOAST BLOCK
# ---------------------------
User-agent: *
Disallow:

Sitemap: http://mrmrsenglish.com/sitemap_index.xml
# ---------------------------
# END YOAST BLOCK


In [3]:
url = 'https://mrmrsenglish.com/different-shape-names/'
response = requests.get(url)  
print(response.content[:100]) 

b'<!DOCTYPE html>\n<html lang="en-US" class="no-js no-svg">\n\n<head><meta http-equiv="Content-Type" cont'


In [4]:
soup = BeautifulSoup(response.content, 'html.parser')

selectors = [
    '#post-3072 > div.entry-content > ul:nth-child(8)',  
    '#post-3072 > div.entry-content > ul:nth-child(13)', 
    '#post-3072 > div.entry-content > div.flex-1.overflow-hidden > div > div > div > div > div > div > div > div > div > div > div > ul',  
    '#post-3072 > div.entry-content > ul:nth-child(22)'  
]

parent = os.path.abspath(os.path.join(os.getcwd(), '..')) 
path = os.path.join(parent, 'data', 'scraped_shapes.csv')

with open(path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Index', 'Shape', 'Description', 'Image URL'])
    index = 1

    for selector in selectors:
        shapes = soup.select_one(selector)

        for li in shapes.find_all('li'):
            shape = li.strong.text 
            description = li.text.split(':')[1].strip() 
            img = li.find('img')
            img_url = img.get('data-lazy-src') or img.get('src')

            print(f"Index: {index}, Shape: {shape}, Description: {description}, Image URL: {img_url}")

            writer.writerow([index, shape, description, img_url])
            
            index += 1

Index: 1, Shape: Circle, Description: A round shape with all points on its boundary equidistant from its center., Image URL: http://mrmrsenglish.com/wp-content/uploads/2024/02/1-24.png
Index: 2, Shape: Square, Description: A quadrilateral with all sides of equal length and all angles at 90 degrees., Image URL: http://mrmrsenglish.com/wp-content/uploads/2024/02/2-24.png
Index: 3, Shape: Triangle, Description: A three-sided and three-angled polygon., Image URL: http://mrmrsenglish.com/wp-content/uploads/2024/02/3-24.png
Index: 4, Shape: Rectangle, Description: A quadrilateral with opposite sides of equal length and all angles at 90 degrees., Image URL: http://mrmrsenglish.com/wp-content/uploads/2024/02/4-24.png
Index: 5, Shape: Pentagon, Description: A polygon with five sides and five angles., Image URL: http://mrmrsenglish.com/wp-content/uploads/2024/02/5-23.png
Index: 6, Shape: Hexagon, Description: Six-sided and six-angled polygon., Image URL: http://mrmrsenglish.com/wp-content/upload

In [5]:
parent = os.path.abspath(os.path.join(os.getcwd(), '..'))  
path = os.path.join(parent, 'data', 'scraped_shapes.csv')
save = os.path.join(parent, 'data', 'shapes_with_colour.csv')

df = pd.read_csv(path)
img_url = df.columns[-1]
colour = []

for url in df[img_url]:
    response = requests.get(url)
    img = BytesIO(response.content)
    ct = ColorThief(img)
    dominant = ct.get_color(quality=1)
    colour.append(dominant)

df[['R', 'G', 'B']] = pd.DataFrame(colour, index=df.index)

df.to_csv(save, index=False)